# Ejercicio: Ajuste de una curva polinómica

En este ejercicio, veremos un tipo diferente de regresión llamada regresión polinómica. A diferencia de la regresión lineal, que modela las relaciones como líneas rectas, la regresión polinómica modela las relaciones como curvas.

Recordemos que en nuestro ejercicio anterior la relación entre la temperatura_del_núcleo y el contenido_proteico_de_la_última_comida no podía explicarse adecuadamente utilizando una línea recta. En este ejercicio, utilizaremos la regresión polinómica para ajustar una curva a los datos.

## Visualización de datos
Empecemos este ejercicio cargando y echando un vistazo a nuestros datos.

In [1]:
import pandas

#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-illness.csv

#Import the data from the .csv file
dataset = pandas.read_csv('doggy-illness.csv', delimiter="\t")

#Let's have a look at the data
dataset

,male,attended_training,age,body_fat_percentage,core_temperature,ate_at_tonys_steakhouse,needed_intensive_care,protein_content_of_last_meal
0,0,1,6.9,38,38.423169,0,0,7.66
1,0,1,5.4,32,39.015998,0,0,13.36
2,1,1,5.4,12,39.148341,0,0,12.90
3,1,0,4.8,23,39.060049,0,0,13.45
4,1,0,4.8,15,38.655439,0,0,10.53
...,...,...,...,...,...,...,...,...
93,0,0,4.5,38,37.939942,0,0,7.35
94,1,0,1.8,11,38.790426,1,1,12.18
95,0,0,6.6,20,39.489962,0,0,15.84
96,0,0,6.9,32,38.575742,1,1,9.79


## Regresión lineal simple 
Refresquemos rápidamente nuestra memoria realizando la misma regresión lineal simple que hicimos en el ejercicio anterior usando las columnas temperatura y contenido_proteína_de_la_última_comida del conjunto de datos.

In [2]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

# Perform linear regression. This method takes care of
# the entire fitting procedure for us.
simple_formula = "core_temperature ~ protein_content_of_last_meal"
simple_model = smf.ols(formula = simple_formula, data = dataset).fit()

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             trendline=lambda x: simple_model.params[1] * x + simple_model.params[0])

Obsérvese que la relación entre las dos variables no es realmente lineal. Si se observa el gráfico, queda bastante claro que los puntos tienden más hacia un lado de la línea, especialmente en el caso de los valores más altos de temperatura central y contenido_proteico_de_la_última_comida. Puede que una línea recta no sea la mejor forma de describir esta relación.

Echemos un vistazo rápido a la puntuación R-cuadrado del modelo:

In [3]:
print("R-squared:", simple_model.rsquared)

R-squared: 0.9155158150005704


Se trata de una puntuación R-cuadrado bastante razonable, ¡pero veamos si podemos obtener una aún mejor!

Regresión polinómica simple
Esta vez vamos a ajustar una regresión polinómica simple. De forma similar a una regresión lineal simple, una regresión polinómica simple modela la relación entre una etiqueta y una única característica. A diferencia de una regresión lineal simple, una regresión polinómica simple puede explicar relaciones que no son simples líneas rectas.

En nuestro ejemplo, vamos a utilizar un polinomio de tres parámetros.

In [4]:
# Perform polynomial regression. This method takes care of
# the entire fitting procedure for us.
polynomial_formula = "core_temperature ~ protein_content_of_last_meal + I(protein_content_of_last_meal**2)"
polynomial_model = smf.ols(formula = polynomial_formula, data = dataset).fit()

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             # Our trendline is the equation for the polynomial
                             trendline=lambda x: polynomial_model.params[2] * x**2 + polynomial_model.params[1] * x + polynomial_model.params[0])


Eso ya se ve mucho mejor. Confirmemos echando un vistazo rápido a la puntuación R-Squared:

In [ ]:
print("R-squared:", polynomial_model.rsquared)

Es una puntuación R-cuadrado_ mejor que la obtenida con el modelo anterior: ¡genial! Ahora podemos decirle con confianza a nuestro veterinario que dé prioridad a los perros que comieron una dieta rica en proteínas la noche anterior.

Vamos a representar nuestro modelo como un gráfico 3D. Veremos  
X y X2
como dos parámetros separados. Observa que si giras el visual lo justo, nuestro modelo de regresión sigue siendo un plano. Por eso los modelos polinómicos siguen considerándose modelos lineales.

In [5]:
import numpy as np
fig = graphing.surface(
    x_values=np.array([min(dataset.protein_content_of_last_meal), max(dataset.protein_content_of_last_meal)]),
    y_values=np.array([min(dataset.protein_content_of_last_meal)**2, max(dataset.protein_content_of_last_meal)**2]),
    calc_z=lambda x,y: polynomial_model.params[0] + (polynomial_model.params[1] * x) + (polynomial_model.params[2] * y),
    axis_title_x="x",
    axis_title_y="x2",
    axis_title_z="Core temperature"
)
# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.protein_content_of_last_meal, y=dataset.protein_content_of_last_meal**2, z=dataset.core_temperature, mode='markers')
fig.show()

## Extrapolación 

Veamos qué ocurre si extrapolamos nuestros datos. Nos gustaría ver si los perros que comieron comidas aún más ricas en proteínas enfermarán aún más.

Empecemos con la regresión lineal. Podemos establecer sobre qué rango nos gustaría extrapolar nuestros datos utilizando el argumento x_range en la función de trazado. Extrapolemos sobre el rango [0,100].

In [6]:
# Show an extrapolated graph of the linear model
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             # We extrapolate over the following range
                             x_range = [0,100],
                             trendline=lambda x: simple_model.params[1] * x + simple_model.params[0])

A continuación, extrapolamos la regresión polinomial sobre el mismo rango:

In [7]:
# Show an extrapolated graph of the polynomial model
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             # We extrapolate over the following range
                             x_range = [0,100],
                             trendline=lambda x: polynomial_model.params[2] * x**2 + polynomial_model.params[1] * x + polynomial_model.params[0])


Estos dos gráficos predicen dos cosas muy diferentes.

La regresión polinómica extrapolada espera que la temperatura central disminuya, mientras que la regresión lineal extrapolada espera que la temperatura central aumente. Un rápido vistazo a los gráficos obtenidos en el ejercicio anterior confirma que deberíamos esperar que la temperatura central aumente a medida que aumenta el contenido proteico de la última comida, no que disminuya.

En general, no se recomienda extrapolar a partir de una regresión polinómica a menos que se tenga una razón a priori para hacerlo (lo que sólo ocurre muy raramente, por lo que es mejor pecar de precavido y no extrapolar nunca a partir de regresiones polinómicas).

Resumen
Hemos cubierto los siguientes conceptos en este ejercicio:

Construir modelos de regresión lineal simple y de regresión polinómica simple.
Comparar el rendimiento de ambos modelos trazándolos y observando los valores de R-cuadrado.
Extrapolar los modelos sobre un rango más amplio de valores.